# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port lincoln,-34.7333,135.8667,291.30,70,83,6.06,AU,1711583087
1,1,waitangi,-43.9535,-176.5597,286.94,76,100,1.79,NZ,1711583090
2,2,sitka,57.0531,-135.3300,284.09,58,40,3.60,US,1711583087
3,3,ribeira grande,38.5167,-28.7000,283.58,77,100,10.28,PT,1711583097
4,4,margaret river,-33.9500,115.0667,291.13,67,0,1.96,AU,1711583096


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",  
    geo = True,
    tiles = "OSM",
    s = "Humidity"
  )

city_map_plot

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_city_data_df = city_data_df.loc[city_data_df["Cloudiness"] == 0]
# Drop any rows with null values
# YOUR CODE HERE
filtered_city_data_df = filtered_city_data_df.dropna()
# Display sample data
filtered_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,margaret river,-33.9500,115.0667,291.13,67,0,1.96,AU,1711583096
11,11,brookings,44.3114,-96.7984,270.15,63,0,6.17,US,1711583213
50,50,saipan,15.1355,145.7010,302.54,1,0,4.12,MP,1711583113
71,71,carnarvon,-24.8667,113.6333,299.57,40,0,4.63,AU,1711583089
73,73,galliano,29.4422,-90.2992,294.15,48,0,5.14,US,1711583225


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[["City", "Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ''
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
4,margaret river,AU,-33.9500,115.0667,67,
11,brookings,US,44.3114,-96.7984,63,
50,saipan,MP,15.1355,145.7010,1,
71,carnarvon,AU,-24.8667,113.6333,40,
73,galliano,US,29.4422,-90.2992,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 20
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame

    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params = {
    "filter": f"circle:{longitude},{latitude},{radius}",
    "limit": limit,
    "bias": f"proximity:{longitude},{latitude}",
    "apiKey":geoapify_key,  
    "categories": 'accommodation.hotel'
}
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
margaret river - nearest hotel: Margaret River Hotel
brookings - nearest hotel: Quality Inn
saipan - nearest hotel: Chalan Kanoa Beach Hotel
carnarvon - nearest hotel: No hotel found
galliano - nearest hotel: Fairfield Inn & Suites by Marriott Cut Off-Galliano
taoudenni - nearest hotel: No hotel found
calama - nearest hotel: Apart Hotel Geotel Calama
lebu - nearest hotel: No hotel found
nica - nearest hotel: Hôtel du Centre
camayenne - nearest hotel: Palm Camayenne
bykovo - nearest hotel: Санаторий "Удельная"
colonia - nearest hotel: Wasserturm Hotel Cologne
qobustan - nearest hotel: No hotel found
knoxville - nearest hotel: Hilton Knoxville
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
hamilton - nearest hotel: North Vista Manor
nueva rosita - nearest hotel: Centro Express
lamar - nearest hotel: 1st St Lofts
tazacorte - nearest hotel: App Leyma
ixtapa - nearest hotel: Ma. Cristina
tlahualilo de zaragoza - nearest hotel: No hotel found
geraldton - n

,City,Country,Lat,Lng,Humidity,Hotel Name
4,margaret river,AU,-33.9500,115.0667,67,Margaret River Hotel
11,brookings,US,44.3114,-96.7984,63,Quality Inn
50,saipan,MP,15.1355,145.7010,1,Chalan Kanoa Beach Hotel
71,carnarvon,AU,-24.8667,113.6333,40,No hotel found
73,galliano,US,29.4422,-90.2992,48,Fairfield Inn & Suites by Marriott Cut Off-Gal...
74,taoudenni,ML,22.6783,-3.9836,13,No hotel found
85,calama,CL,-22.4667,-68.9333,48,Apart Hotel Geotel Calama
92,lebu,CL,-37.6167,-73.6500,90,No hotel found
95,nica,FR,43.7031,7.2661,65,Hôtel du Centre
102,camayenne,GN,9.5350,-13.6878,74,Palm Camayenne


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_city_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",  
    geo = True,
    tiles = "OSM",
    s = "Humidity",
    hover_cols = "all"
 )

hotel_city_map_plot

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,index,City,Country,Hotel Name)